In [2]:
import nltk
import os

In [3]:
import pickle
import csv

In [23]:
# Check for file

if os.path.exists("./model.pickle"):
    #if it exists,load the data
    model = open('./model.pickle','rb')
    classifier = pickle.load(model)
    model.close()
    
    #then, display 5 most informative features
    classifier.show_most_informative_features(5)
else:
    # If it does not exist, Train a model
    
    
    # 1. Load the dataset from the csv file
    dataset = []

    with open('./dataset.csv', encoding= "utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            dataset.append((row['text'],row['label']))
    #                           ^  its the header of the file
    
    # 2. Preprocess the data
        # Tokenize the data
    from nltk.tokenize import word_tokenize
        
    list_words = []

    for sentence, label in dataset:
        words = word_tokenize(sentence)
        list_words.extend(words)
        
        # Remove stopwords, symbol & numbers
    from nltk.corpus import stopwords

    def clean_text(input):
        # make lower
        input = [word.lower() for word in input]
        # remove symbols
        input = [word for word in input if word.isalpha()]
        # remove stopwords
        stop_words = set(stopwords.words('english'))
        input = [word for word in input if not word in stop_words]
        # remove numbers
        input = [word for word in input if not word.isdigit()] ##! isnumeric() can also work
        # remove punctuation
        from string import punctuation
        input = [word for word in input if not word in punctuation]
        return input
    
        #clean data w/ func
    list_words = clean_text(list_words)
        
        # lemmatize / stem the data
        
        # lemmatize:
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
        
    def lemmatize(input):
        lemmas = [lemmatizer.lemmatize(word) for word in input]
        return lemmas
    
    list_words = lemmatize(list_words)
    
    
    # Model training (?)
        # A. use nltk.FreqDist to get the most common words
    from nltk.probability import  FreqDist
    fd = FreqDist(list_words)
    # use the 1000 most common words as features
    list_words = [word for word, count in fd.most_common(1000)]
    
        # B. Create a feature set
    processed_dataset = []
    for sentence, label in dataset:
        words = word_tokenize(sentence)
        words = clean_text(words)
        words = lemmatize(words)
        dict_features = {}
        for feature in list_words:
            dict_features[feature] = (feature in words)
        processed_dataset.append((dict_features,label))
        
        # C. Split the dataset into training and testing
    import random
    
    random.shuffle(processed_dataset)
    split_index = int(len(processed_dataset)* 0.7)
    training_data = processed_dataset[:split_index]
    testing_data = processed_dataset[split_index:]
    
        # D. Train the model
    from nltk.classify import NaiveBayesClassifier, accuracy
    classifier = NaiveBayesClassifier.train(training_data)
        
    
    
        # 3. Classify reviews as either Positive or Negative
    test_input = "Absolutely blown away by the food here! Service was top-notch, and the ambiance made it feel like I was dining in Italy. I will definitely be coming back!"
    test_input_token = word_tokenize(test_input)
    test_input_token = clean_text(test_input_token)
    test_input_token = lemmatize(test_input_token)
    
    test_input_result = classifier.classify(FreqDist(test_input_token))
    print(test_input_result)
    
        # 4.  Show the 5 most informative features and the training accuracy
    classifier.show_most_informative_features(5)    
    print(accuracy(classifier,testing_data))
    
        # 5. Save the model as model.pickle
    model = open('./model.pickle','wb')
    pickle.dump(classifier,model)
    model.close()    

positive
Most Informative Features
                terrible = True           negati : positi =     10.2 : 1.0
                    felt = True           negati : positi =      7.7 : 1.0
                horrible = True           negati : positi =      7.7 : 1.0
                   issue = True           negati : positi =      7.7 : 1.0
                    fish = True           negati : positi =      7.1 : 1.0
0.7696969696969697


In [5]:
##! check how many in fd

print(len(fd))

##! how to see all of fd

for word, frequency in fd.most_common():
    print(f'{word}: {frequency}')

food: 338
place: 312
good: 288
time: 238
like: 225
one: 214
get: 214
great: 210
would: 204
service: 190
back: 177
go: 171
restaurant: 158
really: 145
u: 132
got: 131
could: 125
also: 118
never: 115
even: 112
people: 111
make: 105
minute: 100
much: 99
went: 98
nice: 98
order: 97
ordered: 96
come: 95
chicken: 95
pretty: 93
experience: 91
came: 91
two: 90
best: 89
little: 89
drink: 89
well: 88
night: 88
price: 87
know: 86
better: 86
table: 86
room: 84
going: 84
take: 84
made: 82
want: 81
first: 81
said: 80
ever: 80
say: 80
thing: 78
bar: 78
lot: 76
staff: 75
love: 75
think: 75
day: 74
always: 72
menu: 72
wait: 69
year: 68
sure: 67
took: 67
told: 67
beer: 66
need: 66
asked: 66
cheese: 66
still: 65
bad: 65
lunch: 65
around: 64
definitely: 64
sauce: 64
hotel: 64
friendly: 63
though: 63
meal: 63
way: 63
something: 62
give: 62
check: 62
star: 62
new: 61
find: 60
try: 60
last: 59
waitress: 59
since: 59
review: 58
friend: 58
hour: 58
bit: 58
right: 57
next: 56
pittsburgh: 56
work: 56
taste: 56
c

In [6]:
print (list_words)

['contrary', 'review', 'zero', 'complaint', 'service', 'price', 'getting', 'tire', 'service', 'past', 'year', 'compared', 'experience', 'place', 'like', 'pep', 'boy', 'guy', 'experienced', 'know', 'one', 'place', 'feel', 'like', 'taken', 'advantage', 'gender', 'auto', 'mechanic', 'notorious', 'capitalizing', 'ignorance', 'car', 'sucked', 'bank', 'account', 'dry', 'service', 'road', 'coverage', 'well', 'explained', 'let', 'decide', 'renovated', 'waiting', 'room', 'look', 'lot', 'better', 'previous', 'year', 'last', 'summer', 'appointment', 'get', 'new', 'tire', 'wait', 'super', 'long', 'time', 'also', 'went', 'week', 'fix', 'minor', 'problem', 'tire', 'put', 'free', 'next', 'morning', 'issue', 'called', 'complain', 'even', 'apologize', 'frustrated', 'never', 'going', 'back', 'seem', 'overpriced', 'friendly', 'staff', 'starbucks', 'fair', 'get', 'anywhere', 'else', 'sometimes', 'line', 'get', 'long', 'food', 'good', 'unfortunately', 'service', 'hit', 'miss', 'main', 'issue', 'seems', 'ki

# 2. Application Menu

In [44]:
# Make a menu with 3 options

review = ""

while True:
    print("1. Enter a review")
    print("2. Analyze Your Review")
    print("3. Exit")
    
    ##! enable for debugging
    print("Curr stored input:" + review)
    
    choice = input("Enter your choice: ")

    
    if choice == '1':
        review = input("Enter your review: ")
        
        print("Sentence input successful")
    elif choice == '2':
        # If no review has been entered, return to the menu
        if review == "":
            print("No review has been entered. Please enter a review first.")
            continue
        # Preprocess the review
        else:
            # Make a copy of the review for analysis
            review_step2 = review
            
            # Remove unwanted characters
            review_step2 = word_tokenize(review_step2)
            # Remove symbols & numbers
            review_step2 = [word for word in review_step2 if word.isalpha()]
            review_step2 = [word for word in review_step2 if not word.isdigit()]
            
            # Display POS tagging
            from nltk.tag import pos_tag
            tagged_review = pos_tag(review_step2)
            print("Tagged review:", tagged_review)
        
            # Display synonyms and antonyms for each word
            from nltk.corpus import wordnet
            
            for word in review_step2:
                synonyms = []
                antonyms = []
                for syn in wordnet.synsets(word):
                    for lemma in syn.lemmas():
                        synonyms.append(lemma.name())
                        if lemma.antonyms():
                            antonyms.append(lemma.antonyms()[0].name())
                print(word + ":")
                print("Synonyms:", set(synonyms) if synonyms else "-")
                print("Antonyms:", set(antonyms) if antonyms else "-")
            
            # Do sentiment analysis
            review_step2 = clean_text(review_step2)
            review_step2 = lemmatize(review_step2)
            
            result = classifier.classify(FreqDist(review_step2))
            print("The review is marked as", result)
            
    elif choice == '3':
        print("Exiting program...")
        break
    else:
        print("Invalid choice. Please try again.")

1. Enter a review
2. Analyze Your Review
3. Exit
Curr stored input:
Sentence input successful
1. Enter a review
2. Analyze Your Review
3. Exit
Curr stored input:i love the food
Tagged review: [('i', 'NN'), ('love', 'VBP'), ('the', 'DT'), ('food', 'NN')]
i:
Synonyms: {'i', 'I', 'single', 'unity', 'atomic_number_53', 'ace', 'ane', 'iodine', 'iodin', '1', 'one'}
Antonyms: -
love:
Synonyms: {'make_love', 'bonk', 'lovemaking', 'love', 'fuck', 'bang', 'have_intercourse', 'love_life', 'roll_in_the_hay', 'hump', 'beloved', 'do_it', 'get_laid', 'eff', 'get_it_on', 'passion', 'jazz', 'make_out', 'honey', 'dearest', 'dear', 'have_it_off', 'have_it_away', 'know', 'making_love', 'bed', 'sleep_with', 'sleep_together', 'enjoy', 'be_intimate', 'screw', 'erotic_love', 'have_a_go_at_it', 'sexual_love', 'have_sex', 'lie_with'}
Antonyms: {'hate'}
the:
Synonyms: -
Antonyms: -
food:
Synonyms: {'intellectual_nourishment', 'food_for_thought', 'nutrient', 'food', 'solid_food'}
Antonyms: -
The review is marked 

84
